In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wfdb
import socket

from pylab import rcParams
rcParams['figure.figsize'] = 20, 15
rcParams.update({'font.size': 25})

# determine the paths based off the computer name
hostname=socket.gethostname()

if hostname=='alistair-pc70':
    data_path = '/data/challenge-2015/data/'
    ann_path = '/data/challenge-2015/ann/'
else:
    data_path = 'sample_data/challenge_training_data/'
    ann_path = 'sample_data/challenge_training_data_ann/'
    
# hard-code some colors
col = np.asarray([[228,26,28],
[55,126,184],
[77,175,74],
[152,78,163],
[255,127,0],
[255,255,51],
[166,86,40],
[247,129,191]])/256.0

%matplotlib inline

In [ ]:
# define start time, end time, record name, and annotators
start_time = (4*60+50)
end_time = (5*60)
fs = 250
sample_name = 'v729l'
ann0_name = 'gqrs0'
ann1_name = 'gqrs1'

# choose the lead to plot (annotations are generated off the first lead)
channel = 0

# based on the start time, calculate the sample start/end
start = start_time * fs
end = end_time * fs

# load the data
sig, fields = wfdb.rdsamp(data_path + sample_name)

# create a time vector
t = np.linspace(0,sig.shape[0],sig.shape[0])

ann0 = wfdb.rdann(ann_path + sample_name, ann0_name,
                  sampfrom = start, sampto = end)

ann1 = wfdb.rdann(ann_path + sample_name, ann1_name,
                      sampfrom = start, sampto = end)


# plot the time series
plt.figure(figsize=[16,10])
plt.plot(t[start:end]/fs, sig[start:end,channel], '-',
         color=col[1],linewidth=2,
         label=fields['signame'][channel])

# plot the sample annotation
plt.plot(t[ann0[0]]/fs, sig[ann0[0],channel],
         markeredgecolor=col[0], markerfacecolor=col[0],
         linestyle='none',linewidth=3,
         marker='o',markersize=12,
         label=ann0_name)

# plot the old gqrs
plt.plot(t[ann1[0]]/fs, sig[ann1[0],channel],
         markeredgecolor=col[2], markerfacecolor=col[2],
         linestyle='none',linewidth=3,
         marker='^',markersize=12,
         label=ann1_name)

plt.xlabel('Time (seconds)',fontsize=16)
plt.legend(fontsize=16)
plt.grid()
plt.show()

In [ ]:
# define start time, end time, record name, and annotators
start_time = (4*60+50)
end_time = (5*60)
fs = 250
sample_name = 'v729l'
ann0_name = 'jqrs1'
ann1_name = 'gqrs1'

# choose the lead to plot (annotations are generated off the first lead)
channel = 1

# based on the start time, calculate the sample start/end
start = start_time * fs
end = end_time * fs

# load the data
sig, fields = wfdb.rdsamp(data_path + sample_name)

# create a time vector
t = np.linspace(0,sig.shape[0],sig.shape[0])

ann0 = wfdb.rdann(ann_path + sample_name, ann0_name,
                  sampfrom = start, sampto = end)

ann1 = wfdb.rdann(ann_path + sample_name, ann1_name,
                      sampfrom = start, sampto = end)


# plot the time series
plt.figure(figsize=[16,10])
plt.plot(t[start:end]/fs, sig[start:end,channel], '-',
         color=col[1],linewidth=2,
         label=fields['signame'][channel])

# plot the sample annotation
plt.plot(t[ann0[0]]/fs, sig[ann0[0],channel],
         markeredgecolor=col[0], markerfacecolor=col[0],
         linestyle='none',linewidth=3,
         marker='o',markersize=12,
         label=ann0_name)

# plot the old gqrs
plt.plot(t[ann1[0]]/fs, sig[ann1[0],channel],
         markeredgecolor=col[2], markerfacecolor=col[2],
         linestyle='none',linewidth=3,
         marker='^',markersize=12,
         label=ann1_name)

plt.xlabel('Time (seconds)',fontsize=16)
plt.legend(fontsize=16)
plt.grid()
plt.show()

In [ ]:
import scipy
def hilbert_transform(x, fs, f_low, f_high, demod=False):
    N = len(x)
    f = scipy.fftpack.fft(x, n=N)
    i_high = int(np.floor(float(f_high)/fs*N))
    i_low = int(np.floor(float(f_low)/fs*N))
    win = scipy.signal.hamming( i_high - i_low )
    
    f[0:i_low] = 0
    f[i_low:i_high] = f[i_low:i_high]*win
    f[i_high+1:] = 0
    
    if demod==True:
        # demodulate the signal, i.e. shift the freq spectrum to 0
        i_mid = int( np.floor((i_high+i_low)/2.0) )
        f = np.concatenate( [f[i_mid:i_high], np.zeros(len(f)-(i_high-i_low)), f[i_low:i_mid] ]  )
        
    return 2*np.abs(scipy.fftpack.ifft(f, n=N))

In [ ]:

import matplotlib.patches as patches

fs=250
f_low=5
f_high=25
x = sig[start:end,1]

fg, ax = plt.subplots(3, sharex=True, sharey=True, figsize=[16,10])

# set y limits and x limits (shared)
ylim = [-10, 10]
ax[0].set_ylim([-10, 10])
ax[0].set_xlim([0, fs])

N = len(x)
f = scipy.fftpack.fft(x, n=N)
i_high = np.floor(float(f_high)/fs*N)
i_low = np.floor(float(f_low)/fs*N)
i_mid = np.floor( (i_low+i_high) / 2.0 )

i_high = int(i_high)
i_low = int(i_low)
i_mid = int(i_mid)
win = scipy.signal.hamming( i_high - i_low )
half_win = int(np.floor(len(win)/2))
f_xi = np.linspace(0, fs, len(f))


ax[0].plot(f_xi, f, color=col[0])
ax[0].add_patch( patches.Rectangle( (f_xi[i_low], ylim[0]), # lower left x, y
                                   f_xi[i_mid]-f_xi[i_low], # width
                                   np.diff(ylim), # height
           alpha=0.5, facecolor=col[5]) )
ax[0].add_patch( patches.Rectangle( (f_xi[i_mid], ylim[0]), # lower left x, y
                                   f_xi[i_high]-f_xi[i_mid], # width
                                   np.diff(ylim), # height
           alpha=0.5, facecolor=col[4]) )

f_out = np.copy(f)
f_out[-half_win:] = f_out[i_low:i_mid]
f_out[0:half_win] = f_out[i_mid:i_high]
f_out[half_win:-half_win] = 0
y_out_nofilt = 2*np.abs(scipy.fftpack.ifft(np.copy(f_out), n=N))

# plot signal
ax[1].plot(f_xi, f_out, color=col[2])

# plot location of new window
ax[1].add_patch( patches.Rectangle( (f_xi[0], ylim[0]), # lower left x, y
                                   f_xi[i_high]-f_xi[i_mid], # width
                                   np.diff(ylim), # height
           alpha=0.5, facecolor=col[4]) )
ax[1].add_patch( patches.Rectangle( (f_xi[-half_win], ylim[0]), # lower left x, y
                                   f_xi[i_mid]-f_xi[i_low], # width
                                   np.diff(ylim), # height
           alpha=0.5, facecolor=col[5]) )

# apply the hamming window
f_out = np.copy(f)
f_out[0:] = 0
f_out[-half_win:] = f[i_low:i_mid]
f_out[0:half_win] = f[i_mid:i_high]
f_out[-half_win:] = f_out[-half_win:]*win[0:half_win]
f_out[0:half_win] = f_out[0:half_win]*win[half_win:]
y_out_h = 2*np.abs(scipy.fftpack.ifft(f_out, n=N))

# plot signal
ax[2].plot(f_xi, f_out, color=col[3])

# plot location of new window
ax[2].add_patch( patches.Rectangle( (f_xi[0], ylim[0]), # lower left x, y
                                   f_xi[i_high]-f_xi[i_mid], # width
                                   np.diff(ylim), # height
           alpha=0.5, facecolor=col[4]) )
ax[2].add_patch( patches.Rectangle( (f_xi[-half_win], ylim[0]), # lower left x, y
                                   f_xi[i_mid]-f_xi[i_low], # width
                                   np.diff(ylim), # height
           alpha=0.5, facecolor=col[5]) )

# plot hamming windows
ax[2].plot( f_xi[0:half_win], 10*win[half_win:], '-', linewidth=2,color=col[6] )
ax[2].plot( f_xi[-half_win:], 10*win[0:half_win], '-', linewidth=2,color=col[6] )


# straight hilbert
f = scipy.fftpack.fft(x, n=N)
f[0:i_low] = 0
f[i_low:i_high] = f[i_low:i_high]*win
f[i_high+1:] = 0
y_out = 2*np.abs(scipy.fftpack.ifft(f, n=N))


# new figure
plt.figure(figsize=[16,10])
t = np.arange(0, len(x), 1, dtype=float) / fs
plt.plot(t, x, '-', color=col[0], label = 'original')
plt.plot(t, y_out_nofilt, '-', linewidth=2, color=col[7], label='zeroed + shifted')
plt.plot(t, y_out_h, '-', linewidth=4, color=col[3], label='zeroed + shifted + filtered (Hilbert)')
plt.plot(t, y_out, '--', linewidth=2, color=col[6], label='test')
plt.legend()
plt.show()

In [ ]:
from scipy import signal

# define start time, end time, record name, and annotators
start_time = (4*60+50)
end_time = (5*60)
fs = 250
sample_name = 'v729l'
ann0_name = 'jqrs1'
ann1_name = 'gqrs1'

# choose the lead to plot
channel = 1

# based on the start time, calculate the sample start/end
start = start_time * fs
end = end_time * fs

# load the data
sig, fields = wfdb.rdsamp(data_path + sample_name)

# create a time vector
t = np.linspace(0,sig.shape[0],sig.shape[0])

# load the annotations
ann0 = wfdb.rdann(ann_path + sample_name, ann0_name,
                  sampfrom = start, sampto = end)

ann1 = wfdb.rdann(ann_path + sample_name, ann1_name,
                      sampfrom = start, sampto = end)

# plot the time series
plt.figure(figsize=[16,10])
plt.plot(t[start:end]/fs, sig[start:end,channel], '-',
         color=col[1],linewidth=2,
         label=fields['signame'][channel])

# calculate and plot the Hilbert transform (weird)
hilb_weird = hilbert_transform(sig[start:end,channel], fs, f_low, f_high, demod=True)
plt.plot(t[start:end]/fs, hilb_weird, '-',
         color=col[6], linewidth=2, alpha=0.5,
         label=fields['signame'][channel] + ' - Hilbert transform (weird)')

# calculate and plot the Hilbert transform
hilb = hilbert_transform(sig[start:end,channel], fs, f_low, f_high, demod=False)
plt.plot(t[start:end]/fs, hilb, '--',
         color=col[7], linewidth=2,
         label=fields['signame'][channel] + ' - Hilbert transform (normal)')

# plot the sample annotation
plt.plot(t[ann0[0]]/fs, sig[ann0[0],channel],
         markeredgecolor=col[0], markerfacecolor=col[0],
         linestyle='none',linewidth=3,
         marker='o',markersize=12,
         label=ann0_name)

# plot the old gqrs
plt.plot(t[ann1[0]]/fs, sig[ann1[0],channel],
         markeredgecolor=col[2], markerfacecolor=col[2],
         linestyle='none',linewidth=3,
         marker='^',markersize=12,
         label=ann1_name)

plt.xlabel('Time (seconds)',fontsize=16)
plt.legend(fontsize=16)
plt.grid()
plt.show()